# 5 - Ejemplos programación RDDs Spark Core

In [1]:
# Load external packages programatically
import os
# packages = "com.databricks:spark-xml_2.11:0.5.0"

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["JAVA_HOME"]

# os.environ["PYSPARK_SUBMIT_ARGS"] = (
#     "--packages {0} pyspark-shell".format(packages)
# )

'/usr/lib/jvm/java-8-openjdk-amd64'

In [20]:
import pyspark
from pyspark.sql import SparkSession
spark = (SparkSession.builder
    .master("local[*]")
    .config("spark.driver.cores", 1)
    .appName("Simple Word Count")
    .getOrCreate() )
sc = spark.sparkContext

In [21]:
print(spark)
print(sc)

<SparkContext master=local[*] appName=Simple Word Count>


In [22]:
spark

## 5.1 Operaciones de bajo nivel

In [4]:
rdd = sc.parallelize(["#epicfail",  "#hadoop", "#rstats",  "#rstudio", "#rstats", "#spark", "#hadoop", "#hdfs",
"#hadoop",  "#oreilly", "#spark", "#python", "#spark", "#scala", "#spark", "#strataconf", "#strataconf", "#oreilly",
"#spark", "#databricks", "#hadoop", "#hdfs", "#spark",  "#hdfs"], 4)

In [5]:
rdd.getNumPartitions()

4

In [6]:
rdd_distinct = rdd.distinct()
print(rdd_distinct.toDebugString().decode("utf8"))

(4) PythonRDD[5] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[4] at mapPartitions at PythonRDD.scala:133 []
 |  ShuffledRDD[3] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(4) PairwiseRDD[2] at distinct at <ipython-input-6-199515af71a6>:1 []
    |  PythonRDD[1] at distinct at <ipython-input-6-199515af71a6>:1 []
    |  ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262 []


In [7]:
print(rdd_distinct)

PythonRDD[5] at RDD at PythonRDD.scala:53


In [8]:
print(rdd_distinct.collect())

['#hdfs', '#oreilly', '#databricks', '#rstudio', '#spark', '#scala', '#strataconf', '#epicfail', '#hadoop', '#rstats', '#python']


## 5.2 Contador de palabras

In [23]:
rdd_readme = sc.textFile("../data/README.md", 4)

In [24]:
rdd_readme.count()

103

In [25]:
rdd_readme.take(5)

['# Apache Spark',
 '',
 'Spark is a fast and general cluster computing system for Big Data. It provides',
 'high-level APIs in Scala, Java, Python, and R, and an optimized engine that',
 'supports general computation graphs for data analysis. It also supports a']

In [26]:
rdd_word_count =(rdd_readme
.flatMap(lambda line: line.split())
.map(lambda word: (word, 1))
.reduceByKey(lambda x, y: x + y))

In [27]:
print(rdd_word_count.toDebugString().decode("utf8"))

(4) PythonRDD[8] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[7] at mapPartitions at PythonRDD.scala:133 []
 |  ShuffledRDD[6] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(4) PairwiseRDD[5] at reduceByKey at <ipython-input-26-ee2981045352>:1 []
    |  PythonRDD[4] at reduceByKey at <ipython-input-26-ee2981045352>:1 []
    |  ../data/README.md MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []
    |  ../data/README.md HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []


In [28]:
rdd_word_count.take(10)

[('Apache', 1),
 ('It', 2),
 ('provides', 1),
 ('high-level', 1),
 ('in', 6),
 ('Scala,', 1),
 ('Java,', 1),
 ('an', 4),
 ('optimized', 1),
 ('engine', 1)]

In [29]:
rdd_readme.getNumPartitions()

4

In [30]:
rdd_word_count.getNumPartitions()

4

In [17]:
rdd_readme2 = sc.textFile("../data/README.md")

In [18]:
%%timeit -r1 -n1
rdd_word_count =(rdd_readme2
.flatMap(lambda line: line.split())
.map(lambda word: (word, 1))
.reduceByKey(lambda x, y: x + y)
.take(10))

123 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [19]:
spark.stop()